In [2]:
import sys
import torch
import torchvision
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import torchvision.transforms as transforms
import os
import cv2
from torch.utils.data import DataLoader, TensorDataset, Dataset

%load_ext autoreload
%autoreload 2
%matplotlib inline


In [3]:
#hyperparameters 

IMG_SIZE = 224
BATCH_SIZE = 32


In [4]:
#dirs and paths

data_dir = '/data/AlgProj/tct_yaoms/data/tct_0702_preprocess_iou0.1_img_series/the_same_region_labeled/'
csv_path = '/data/AlgProj/niezl/TCT/TCT_Data/Misc/TCT标注_0702/影像标注结果.csv'

csv = pd.read_csv(csv_path)
file_names = os.listdir(data_dir)
df = pd.DataFrame()
df['img_name'] = file_names
# df['label'] = None
# df['img_path'] = None
labels = []
img_paths = []
for idx in df.index:
    img_name = df.loc[idx, 'img_name']
    img_ID = int(img_name.split('_')[0])
#     print(img_ID, '\n-----')
    label = int(csv.loc[csv['影像结果编号']==img_ID, '恶性细胞分类'])
#     print(label)
    img_path = data_dir + img_name
    labels.append(label)
    img_paths.append(img_path)

df['label'] = labels
df['img_path'] = img_paths

,img_name,label,img_path
0,302841_494fd50c722bfdfb0a8381ad942a42e83fafbf2...,1,/data/AlgProj/tct_yaoms/data/tct_0702_preproce...
1,291491_412393f4ad4ef06bd60c0b42fdfde47c234861e...,2,/data/AlgProj/tct_yaoms/data/tct_0702_preproce...
2,283418_8aa1ef362ac94ced2ff9a3973ed17c60594985b...,1,/data/AlgProj/tct_yaoms/data/tct_0702_preproce...
3,302812_ad96c70645d69ff7ab238052f49f15e2ac38d23...,2,/data/AlgProj/tct_yaoms/data/tct_0702_preproce...
4,284624_9ae1ba2e424b0541dd228a29574f9da308663bd...,4,/data/AlgProj/tct_yaoms/data/tct_0702_preproce...


In [64]:
def randomRotation(img_name, times,mode=Image.BICUBIC):
    img_id = img_name.split('_')[0]
    save_dir = '/data/AlgProj/tct_yaoms/data/tct_0702_preprocess_iou0.1_img_series/the_same_region_labeled_dataaugmentation/'
    data_dir = '/data/AlgProj/tct_yaoms/data/tct_0702_preprocess_iou0.1_img_series/the_same_region_labeled/'
    img_path = data_dir + img_name
    img = Image.open(img_path)
    
    if times == 1:
        save_name = img_id + '_' + '.jpg'
        save_path = save_dir + save_name
        img.save(save_path, quality=95)
    else:
        for i in range(times):
            random_angle = np.random.randint(1, 360)
            img = img.rotate(random_angle, mode)
            save_name = img_id + '_'+ str(i) + '.jpg'
            save_path = save_dir + save_name
            img.save(save_path, quality=95)

In [65]:
for idx in df.index:
    img_name = df.loc[idx, 'img_name']
    label = df.loc[idx,'label']
    if label == 1:
        times = 1
    elif label == 2:
        times = 3
    elif label == 3:
        times = 4
    elif label == 4:
        times = 3
    elif label == 5:
        pass
    elif label == 6:
        times = 40
    elif label ==7:
        times = 100
    randomRotation(img_name, times,mode=Image.BICUBIC)

In [68]:
#dirs and paths

data_dir = '/data/AlgProj/tct_yaoms/data/tct_0702_preprocess_iou0.1_img_series/the_same_region_labeled_dataaugmentation/'
csv_path = '/data/AlgProj/niezl/TCT/TCT_Data/Misc/TCT标注_0702/影像标注结果.csv'

csv = pd.read_csv(csv_path)
file_names = os.listdir(data_dir)
tct_df= pd.DataFrame()
tct_df['img_name'] = file_names
# df['label'] = None
# df['img_path'] = None
labels = []
img_paths = []
for idx in tct_df.index:
    img_name = tct_df.loc[idx, 'img_name']
    img_ID = int(img_name.split('_')[0])
#     print(img_ID, '\n-----')
    label = int(csv.loc[csv['影像结果编号']==img_ID, '恶性细胞分类'])
#     print(label)
    img_path = data_dir + img_name
    labels.append(label)
    img_paths.append(img_path)

tct_df['label'] = labels
tct_df['img_path'] = img_paths

In [79]:
bg_df = pd.DataFrame(columns=['img_name', 'label', 'img_path'])
bg_root = '/data/AlgProj/tct_yaoms/data/neg/'
sub_dirs = os.listdir(bg_root)
for sub_dir in sub_dirs:
    cur_dir = bg_root + sub_dir + '/'
    img_names = os.listdir(cur_dir)
    for img_name in img_names:
        img_path = cur_dir + img_name
        bg_df = bg_df.append([{'img_name':img_name, 'label':0, 'img_path':img_path}], ignore_index=True)


In [84]:
all_df = bg_df.loc[:1200,:].append(tct_df, ignore_index=True)
len(all_df)

3934

In [85]:
all_df.to_csv('/data/AlgProj/tct_yaoms/data/tct_0702_preprocess_iou0.1_img_series/iou0.1dataaumentation.csv', index = False)

### 开始训练

In [ ]:
tct_df = pd.read_csv